## 实例化一个小型的卷积神经网络

In [1]:
from keras import layers
from keras import models

Using TensorFlow backend.


卷积神经网络接受形状为 (image_height, image_weight, image_channels)，不包括批量维度。

本例中，图片格式为 28*28 ，通道数为1（灰度图）。

In [2]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Instructions for updating:
Colocations handled automatically by placer.


查看卷积神经网络的架构：

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


每一层的输出都是一个形状为 (height, weight, channels) 的3D张量。宽度和高度两个维度的尺寸通常会随着网络加深而变小。通道数量由传入 Conv2D 层的第一个参数所控制（32或64）。

下一步是将最后的输出张量 [大小为(3, 3, 64)]  输入到Dense层，分类器可以处理1D向量，而当前输出的是3D张量。

## 在卷积神经网络上添加分类器

首先，我们需要将3D输出展平为1D，然后在上面添加几个Dense层。

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

最后一层使用10个分类输出的 softmax 激活，现在的网络机构如下：

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

在进入两个 Dense 层之前，形状 (3, 3, 64) 的输出被展平为形状 (576,) 的向量。

## 在MNIST图像上训练卷积神经网络

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [7]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 20s 336us/step - loss: 0.2027 - acc: 0.9368
Epoch 2/5
60000/60000 [==============================] - 18s 302us/step - loss: 0.0683 - acc: 0.9805
Epoch 3/5
60000/60000 [==============================] - 18s 305us/step - loss: 0.0482 - acc: 0.9868
Epoch 4/5
60000/60000 [==============================] - 19s 309us/step - loss: 0.0385 - acc: 0.9894
Epoch 5/5
60000/60000 [==============================] - 19s 318us/step - loss: 0.0389 - acc: 0.9905


在测试数据上对模型进行评估。

In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 2s 152us/step


0.9884

比简单的全连接模型准确率高。